In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import geopandas as gp
import geoplot as gplt
import geoplot.crs as gcrs
import contextily as ctx
import numpy as np
import shapely as shapely
from shapely.geometry import Polygon
from shapely.ops import transform
import collections
from collections import Counter
from typing import List, Tuple, Dict, Union, Generator, Optional
from pyproj import Transformer, CRS, Proj
import glob
import os
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import advertools as adv
import warnings
import emoji
warnings.filterwarnings('ignore')
import matplotlib as mpl

In [ ]:
"""
Defining constants to be used throughout the program

"""

#create grids based on the custom made eu shapefile

GRID_SIZE_METERS = 100000 
                        
# target projection: Mollweide
EPSG_CODE = 54009
CRS_PROJ = f"esri:{EPSG_CODE}"

# Input projection WGS 84
CRS_WGS = "epsg:4326"

# define Transformer ahead of time
# with xy-order of coordinates
PROJ_TRANSFORMER = Transformer.from_crs(
    CRS_WGS, CRS_PROJ, always_xy=True)

# also define reverse projection
PROJ_TRANSFORMER_BACK = Transformer.from_crs(
    CRS_PROJ, CRS_WGS, always_xy=True)

#projecting the bounds of the eu-shapefile to Mollweide

XMIN = PROJ_TRANSFORMER.transform(-18.729512 , 29.234046)[0]
XMAX = PROJ_TRANSFORMER.transform(39.73858, 29.234046)[0]
YMAX = PROJ_TRANSFORMER.transform(49.59352369, 71.16987838)[1]
YMIN = PROJ_TRANSFORMER.transform(49.59352369, 28.017169)[1]

In [ ]:
# import study area
europe = gp.read_file("Europe_Clipped_BBox.shp")
europe.to_crs(CRS_PROJ, inplace =True)
europe.plot()

In [ ]:
# Let's make some country labels
# Here we find the representative points, copy our original df to a new df, and then set 
# the geometry column to the newly created representative points column 
# (because a GeoPandas df can only have one geometry column)
# I'll also reduce the number of labels so that the basemap isn't too crowded

country_labels = europe.loc[europe['NAME_EN'].isin(["France", "Spain", "Germany", "Algeria", "Poland", "Morocoo",
                                                                  "Libya", "Egypt", "Turkey", "United Kingdom", "Hungary", 
                                                                  "Sweden", "Finland", "Norway", "Ireland", "Greece", "Italy",
                                                                  "Belarus", "Ukraine", "Austria", "Romania", "Denmark",
                                                                 "Russia", "Bulgaria", "Lithuania", "Latvia", "Estonia", 
                                                                  "Iceland", "Morocco"])]
country_labels["rep"] = europe["geometry"].representative_point()
# country_labels = countries_gdf.copy()
country_labels.set_geometry("rep", inplace = True)
ax = europe.plot(figsize = (15, 12), color = "whitesmoke", edgecolor = "lightgrey", linewidth = 0.5)
texts = []

for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
    texts.append(plt.text(x-100000, y, label.upper(), fontsize = 10, color="dimgray")) #shift labels slightly to the west


In [ ]:
df = gp.read_file(r"C:\Users\saman\OneDrive\Documents\Thesis\Data\RawData_Cleaned_Final.geojson")

In [ ]:
# convert to geodf, re-project to mollweide projection
gdf = gp.GeoDataFrame(df,geometry =gp.points_from_xy(df.long,df.lat),crs =4326)
gdf.to_crs(CRS_PROJ,inplace=True)

In [ ]:
# create columns aggregating date information - i'll start just with monthly groupings
gdf['post_publish_date'] = pd.to_datetime(gdf['post_publish_date'])
gdf['Month/Year'] = gdf['post_publish_date'].dt.to_period('M')  # add new column showing timestamps aggregated to monthly intervals

In [ ]:
# now let's create sub-dtasets for typicality calculations, starting with temporal subsets (monthly)
gdf_jan = gdf[gdf['Month/Year'] == '2020-01']
gdf_feb = gdf[gdf['Month/Year'] == '2020-02']
gdf_mar = gdf[gdf['Month/Year'] == '2020-03']
gdf_apr = gdf[gdf['Month/Year'] == '2020-04']
gdf_may = gdf[gdf['Month/Year'] == '2020-05']
gdf_jun = gdf[gdf['Month/Year'] == '2020-06']
gdf_jul = gdf[gdf['Month/Year'] == '2020-07']
gdf_aug = gdf[gdf['Month/Year'] == '2020-08']
gdf_sep = gdf[gdf['Month/Year'] == '2020-09']
gdf_oct = gdf[gdf['Month/Year'] == '2020-10']
# note - no data for november
gdf_dec = gdf[gdf['Month/Year'] == '2020-12']

In [ ]:
# create grid over study area
def create_grids():
    
    """
    Creating polygons based on the grid size
    """
    
    width = GRID_SIZE_METERS
    length = GRID_SIZE_METERS
    cols = list(range(int(np.floor(XMIN)), int(np.ceil(XMAX)), width))
    rows = list(range(int(np.floor(YMIN)), int(np.ceil(YMAX)), length))
    rows.reverse()

    polygons = []
    for x in cols:
         for y in rows:
                # combine to tuple: (x,y, poly)
                # and append to list
                polygons.append(
                    (x, y,
                     Polygon([
                         (x, y),
                         (x+width, y),
                         (x+width, y-length),
                         (x, y-length)])))
    grid = pd.DataFrame(polygons)
        # name columns
    col_labels=['xbin', 'ybin', 'bin_poly']
    grid.columns = col_labels
        # use x and y as index columns
    grid.set_index(['xbin', 'ybin'], inplace=True)
    grid = gp.GeoDataFrame(
            grid.drop(
                columns=["bin_poly"]),
                geometry=grid.bin_poly)
    grid.crs = CRS_PROJ
    return grid,cols,rows

grid,cols,rows = create_grids()

In [ ]:
ybins = np.array(rows)
xbins = np.array(cols)

def get_best_bins(search_values_x, search_values_y,xbins, ybins): 
    """Will return best bin for a lat and lng input
    
    Note: prepare bins and values in correct matching projection
    
    """
    xbins_idx = np.digitize(search_values_x, xbins, right=False)
    ybins_idx = np.digitize(search_values_y, ybins, right=False)
    return (xbins[xbins_idx-1], ybins[ybins_idx-1])


In [ ]:
xbins_match, ybins_match = get_best_bins(
    search_values_x=gdf.geometry.x.to_numpy(),
    search_values_y=gdf.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_jan, ybins_match_jan = get_best_bins(
    search_values_x=gdf_jan.geometry.x.to_numpy(),
    search_values_y=gdf_jan.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_feb, ybins_match_feb = get_best_bins(
    search_values_x=gdf_feb.geometry.x.to_numpy(),
    search_values_y=gdf_feb.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_mar, ybins_match_mar = get_best_bins(
    search_values_x=gdf_mar.geometry.x.to_numpy(),
    search_values_y=gdf_mar.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_apr, ybins_match_apr = get_best_bins(
    search_values_x=gdf_apr.geometry.x.to_numpy(),
    search_values_y=gdf_apr.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_may, ybins_match_may = get_best_bins(
    search_values_x=gdf_may.geometry.x.to_numpy(),
    search_values_y=gdf_may.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_jun, ybins_match_jun = get_best_bins(
    search_values_x=gdf_jun.geometry.x.to_numpy(),
    search_values_y=gdf_jun.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_jul, ybins_match_jul = get_best_bins(
    search_values_x=gdf_jul.geometry.x.to_numpy(),
    search_values_y=gdf_jul.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_aug, ybins_match_aug = get_best_bins(
    search_values_x=gdf_aug.geometry.x.to_numpy(),
    search_values_y=gdf_aug.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_sep, ybins_match_sep = get_best_bins(
    search_values_x=gdf_sep.geometry.x.to_numpy(),
    search_values_y=gdf_sep.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_oct, ybins_match_oct = get_best_bins(
    search_values_x=gdf_oct.geometry.x.to_numpy(),
    search_values_y=gdf_oct.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)
xbins_match_dec, ybins_match_dec = get_best_bins(
    search_values_x=gdf_dec.geometry.x.to_numpy(),
    search_values_y=gdf_dec.geometry.y.to_numpy(),
    xbins=xbins, ybins=ybins)


In [ ]:
#instead of a spatial join, indexes are used to find which hashtag belongs to which grid

#first we'll do one general one:
gdf.loc[:, 'xbins_match'] = xbins_match
gdf.loc[:, 'ybins_match'] = ybins_match
gdf.set_index(['xbins_match', 'ybins_match'], inplace=True)
grid.sort_index(inplace =True)
gdf.sort_index(inplace = True)
common_idx = grid.index.intersection(gdf.index) 

#let's do this for each month
gdf_jan.loc[:, 'xbins_match'] = xbins_match_jan
gdf_jan.loc[:, 'ybins_match'] = ybins_match_jan
# gdf_jan.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_jan.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_jan.sort_index(inplace = True)
common_idx_jan = grid.index.intersection(gdf_jan.index) 

gdf_feb.loc[:, 'xbins_match'] = xbins_match_feb
gdf_feb.loc[:, 'ybins_match'] = ybins_match_feb
# gdf_feb.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_feb.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_feb.sort_index(inplace = True)
common_idx_feb = grid.index.intersection(gdf_feb.index) 

gdf_mar.loc[:, 'xbins_match'] = xbins_match_mar
gdf_mar.loc[:, 'ybins_match'] = ybins_match_mar
# gdf_mar.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_mar.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_mar.sort_index(inplace = True)
common_idx_mar = grid.index.intersection(gdf_mar.index) 

gdf_apr.loc[:, 'xbins_match'] = xbins_match_apr
gdf_apr.loc[:, 'ybins_match'] = ybins_match_apr
# gdf_apr.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_apr.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_apr.sort_index(inplace = True)
common_idx_apr = grid.index.intersection(gdf_apr.index) 

gdf_may.loc[:, 'xbins_match'] = xbins_match_may
gdf_may.loc[:, 'ybins_match'] = ybins_match_may
# gdf_may.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_may.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_may.sort_index(inplace = True)
common_idx_may = grid.index.intersection(gdf_may.index) 

gdf_jun.loc[:, 'xbins_match'] = xbins_match_jun
gdf_jun.loc[:, 'ybins_match'] = ybins_match_jun
# gdf_jun.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_jun.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_jun.sort_index(inplace = True)
common_idx_jun = grid.index.intersection(gdf_jun.index) 

gdf_jul.loc[:, 'xbins_match'] = xbins_match_jul
gdf_jul.loc[:, 'ybins_match'] = ybins_match_jul
# gdf_jul.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_jul.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_jul.sort_index(inplace = True)
common_idx_jul = grid.index.intersection(gdf_jul.index) 

gdf_aug.loc[:, 'xbins_match'] = xbins_match_aug
gdf_aug.loc[:, 'ybins_match'] = ybins_match_aug
# gdf_aug.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_aug.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_aug.sort_index(inplace = True)
common_idx_aug = grid.index.intersection(gdf_aug.index) 

gdf_sep.loc[:, 'xbins_match'] = xbins_match_sep
gdf_sep.loc[:, 'ybins_match'] = ybins_match_sep
# gdf_sep.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_sep.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_sep.sort_index(inplace = True)
common_idx_sep = grid.index.intersection(gdf_sep.index) 

gdf_oct.loc[:, 'xbins_match'] = xbins_match_oct
gdf_oct.loc[:, 'ybins_match'] = ybins_match_oct
# gdf_oct.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_oct.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_oct.sort_index(inplace = True)
common_idx_oct = grid.index.intersection(gdf_oct.index) 

gdf_dec.loc[:, 'xbins_match'] = xbins_match_dec
gdf_dec.loc[:, 'ybins_match'] = ybins_match_dec
# gdf_dec.drop(columns = ['longitude','latitude','geometry'],inplace =True)
gdf_dec.set_index(['xbins_match', 'ybins_match'], inplace=True)
# gdf.dropna(subset = 'hashtag', inplace =True)
grid.sort_index(inplace =True)
gdf_dec.sort_index(inplace = True)
common_idx_dec = grid.index.intersection(gdf_dec.index) 



In [ ]:
# now that the month datasets are prepped, let's calculate typicality for some emojis
def grid_typicality(new_test,idx,typ):    
     
        #calculating frequency for each grid (sub-dataset) 
        counter = Counter()
        new_test.str.split(',').apply(counter.update)
        n_s = counter[EMOJI]
        if (n_s == 0):
            typ.loc[idx,'typicality'] = -1.0
        else:    
            N_s = sum(counter.values())
            F_s = n_s/N_s
            typ.loc[idx,'typicality'] = (F_s - F_t)/F_t 

In [ ]:
#counting the occurence of each emoji in preparation of typicality calculations
count = Counter()  
gdf_jan['emoji generic'].str.split(',').apply(count.update)
gdf_feb['emoji generic'].str.split(',').apply(count.update)
gdf_mar['emoji generic'].str.split(',').apply(count.update)
gdf_apr['emoji generic'].str.split(',').apply(count.update)
gdf_may['emoji generic'].str.split(',').apply(count.update)
gdf_jun['emoji generic'].str.split(',').apply(count.update)
gdf_jul['emoji generic'].str.split(',').apply(count.update)
gdf_aug['emoji generic'].str.split(',').apply(count.update)
gdf_sep['emoji generic'].str.split(',').apply(count.update)
gdf_oct['emoji generic'].str.split(',').apply(count.update)
gdf_dec['emoji generic'].str.split(',').apply(count.update)


In [ ]:
# did I accidentally delete this????
typ = pd.DataFrame(index = common_idx, columns = ['typicality'], data = '') #dummy dataframe to hold the typicality values

In [ ]:
geom = grid.loc[common_idx, "geometry"]
typ_jan = gp.GeoDataFrame(data = typ_jan['typicality'], geometry =geom, crs = CRS_PROJ)
typ_feb = gp.GeoDataFrame(data = typ_feb['typicality'], geometry =geom, crs = CRS_PROJ)
typ_mar = gp.GeoDataFrame(data = typ_mar['typicality'], geometry =geom, crs = CRS_PROJ)
typ_apr = gp.GeoDataFrame(data = typ_apr['typicality'], geometry =geom, crs = CRS_PROJ)
typ_may = gp.GeoDataFrame(data = typ_may['typicality'], geometry =geom, crs = CRS_PROJ)
typ_jun = gp.GeoDataFrame(data = typ_jun['typicality'], geometry =geom, crs = CRS_PROJ)
typ_jul = gp.GeoDataFrame(data = typ_jul['typicality'], geometry =geom, crs = CRS_PROJ)
typ_aug = gp.GeoDataFrame(data = typ_aug['typicality'], geometry =geom, crs = CRS_PROJ)
typ_sep = gp.GeoDataFrame(data = typ_sep['typicality'], geometry =geom, crs = CRS_PROJ)
typ_oct = gp.GeoDataFrame(data = typ_oct['typicality'], geometry =geom, crs = CRS_PROJ)
typ_dec = gp.GeoDataFrame(data = typ_dec['typicality'], geometry =geom, crs = CRS_PROJ)

In [ ]:
# and let's use a nice custom color ramp
typ_cmap = mpl.colors.LinearSegmentedColormap.from_list('beigeblue',['#d9af8c','#FFFFFF','#009999'], N=8)

In [ ]:
monthlist = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "December"]

In [ ]:
# let's test this out with the masked face emoji
EMOJI = '😷'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Mask\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
# let's test this out with the masked face emoji
EMOJI = '😷'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of      Emoji (" + month +")", size =35, y=0.78) 
    # will add the emoji image later since matplotlib doesn't accept them in titles
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Mask\TypicalityMap_BlankTitle_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
# let's test this out with the masked face emoji
EMOJI = '😷'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +"       Emoji (" + month +")", size =35, y=0.78) 
    # will add the emoji image later since matplotlib doesn't accept them in titles
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Mask\TypicalityMap_Space_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '🦠'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
# let's test this out with the masked face emoji
EMOJI = '🦠'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of      Emoji (" + month +")", size =35, y=0.78) # will add the emoji image later
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Mask\TypicalityMap_BlankTitle_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '✊'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\RaisedFist\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
# let's test this out with the masked face emoji
EMOJI = '✊'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of      Emoji (" + month +")", size =35, y=0.78) # will add the emoji image later
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Mask\TypicalityMap_BlankTitle_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '🍺'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
# let's test this out with the masked face emoji
EMOJI = '🍺'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of      Emoji (" + month +")", size =35, y=0.78) # will add the emoji image later
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Mask\TypicalityMap_BlankTitle_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '🌈'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
# let's test this out with the masked face emoji
EMOJI = '🌈'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of      Emoji (" + month +")", size =35, y=0.78) # will add the emoji image later
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Mask\TypicalityMap_BlankTitle_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '👏'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
# let's test this out with the masked face emoji
EMOJI = '👏'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of      Emoji (" + month +")", size =35, y=0.78) # will add the emoji image later
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Mask\TypicalityMap_BlankTitle_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


# Those were the hypothesized emojis with topical consistency, now I'll investigate the emojis with topical consistency that were found to be typical in various countries

In [ ]:
EMOJI = '🤱'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '🏌️‍♂️'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '🕗'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '👩‍⚕️'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '🏥'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '🗳️'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '💣'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '🚁'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")


In [ ]:
EMOJI = '🚁'

emojiname_us = emoji.demojize(str(EMOJI)).replace(":","")
emojiname = emojiname_us.replace("_", " ")
emojiname = emojiname.title()

for month in monthlist:
    
    if month == "January":
        gdf = gdf_jan
        common_idx = common_idx_jan
        typ_gdf = typ_jan
    elif month == "February":
        gdf = gdf_feb
        common_idx = common_idx_feb
        typ_gdf = typ_feb
    elif month == "March":
        gdf = gdf_mar
        common_idx = common_idx_mar
        typ_gdf = typ_mar
    elif month == "April":
        gdf = gdf_apr
        common_idx = common_idx_apr
        typ_gdf = typ_apr
    elif month == "May":
        gdf = gdf_may
        common_idx = common_idx_may
        typ_gdf = typ_may
    elif month == "June":
        gdf = gdf_jun
        common_idx = common_idx_jun
        typ_gdf = typ_jun
    elif month == "July":
        gdf = gdf_jul
        common_idx = common_idx_jul
        typ_gdf = typ_jul
    elif month == "August":
        gdf = gdf_aug
        common_idx = common_idx_aug
        typ_gdf = typ_aug
    elif month == "September":
        gdf = gdf_sep
        common_idx = common_idx_sep
        typ_gdf = typ_sep
    elif month == "October":
        gdf = gdf_oct
        common_idx = common_idx_oct
        typ_gdf = typ_oct
    elif month == "December":
        gdf = gdf_dec
        common_idx = common_idx_dec
        typ_gdf = typ_dec
    
    
    #calculating frequency for total dataset     
    n_t = count[EMOJI]
    N_t = sum(count.values())
    F_t = n_t/N_t

    for idx,midx in enumerate(common_idx): #looping through all the common indexes between the grids and dataframe
        grid_typicality(gdf.loc[midx,"emoji"], common_idx[idx], typ_gdf)

    geom = grid.loc[common_idx, "geometry"]

    base = grid.plot(figsize=(22,28), color='white', alpha=0)
    base.grid(False)
    # Hide axes ticks
    base.axes.xaxis.set_visible(False)
    base.axes.yaxis.set_visible(False)
    # combine with europe geometry
    europe.boundary.plot(ax=base, edgecolor='dimgray', linewidth=0.7, zorder=10) 
    # add labels using adjust text
    for x, y, label in zip(country_labels.geometry.x, country_labels.geometry.y, country_labels["NAME_EN"]):
        texts.append(plt.text(x-100000, y, label.upper(), fontsize = 14, color="dimgray", zorder=10))
    plot = typ_gdf.plot(ax=base, column = 'typicality', colormap=typ_cmap, alpha = 0.85, zorder=5)
    plt.grid(False)
    fig = plot.get_figure()
    fig.suptitle("Typicality of "+ emojiname +" Emoji (" + month +")", size =35, y=0.78)
    fig.savefig(r"C:\Users\saman\OneDrive\Documents\Thesis\Figures\Animations\Microbe\TypicalityMap_" + emojiname_us + "_"+month+".png", dpi=300, bbox_inches = "tight")
